In [13]:

from databaker.framework import *


In [14]:
import pandas as pd

import sys
inputFile = sys.argv[1] #"underlyingdatasheet4.01ipscitizenshipgroupbysexbyagebycountryoflastornextresidence2016.xls"
xl = pd.ExcelFile(inputFile)
sheets = xl.sheet_names


"""
Use databaker to amatch up column index and readable column label
then shift to pandas and incorporate the data
"""
tabs = loadxlstabs(inputFile)
allDataFrames = []

# Get the year from the filename
year = inputFile.split(".xls")[0][-4:]
print(year)

for tab in tabs[-1:]:
    
    rowVars = tab.filter(contains_string("Software Readable Row Label")).shift(1,2)
    rowVars = rowVars.expand(RIGHT).is_not_blank().is_not_whitespace()
    
    rowVarDict = {}
    for r in rowVars:
        rowVarDict.update({int(r.x):r.value})
        skipUntil = r.y
        
    # Use databker to find the end of the sheets
    sheetEnds = tab.filter(contains_string("Source: Office for National Statistics"))
    for sh in sheetEnds:
        lastrow = sh.y -3
    
    
    # Now load the sheet as a dataframe, skipping down to the obs
    for i in range(0, len(sheets)):
        if tab.name == sheets[i]:
            loadSheet = sheets[i]
    
    # Load, skim off the header rows, and replace unwanted column names with _integers (_1, _2 etc)
    # Replace Wanted column names with appropriate databakered rowVar
    oldDf = xl.parse(loadSheet)
    oldDf = oldDf[skipUntil-1:lastrow]
    
    toDelete = []
    newHeaders = []
    for i in range(1, len(oldDf.columns.values)+1):
        if i in rowVarDict.keys():
            newHeaders.append(rowVarDict[i])
        else:
            newHeaders.append("_" + str(i))
            toDelete.append("_" + str(i))
    
    oldDf.columns = newHeaders
    
    # Delete hlding columns _1, _2 etc
    for delMe in toDelete:
        oldDf = oldDf.drop(delMe, axis=1)
        
    oldDf["Time"] = year
    
    allDataFrames.append(oldDf)
    
sourceDf = pd.concat(allDataFrames)
sourceDf[:20]


Loading underlyingdatasheet4.01ipscitizenshipgroupbysexbyagebycountryoflastornextresidence2016.xls which has size 6391296 bytes
Table names: ['Contents and Notes', 'Datasheet 4.01A', 'Datasheet 4.01B', 'Datasheet 4.01C', 'Datasheet 4.01D']
2016


,RESC All EST,RESC All CI,RESC Oman EST,RESC Oman CI,RESC Pakistan EST,RESC Pakistan CI,RESC Palestine EST,RESC Palestine CI,RESC Paraguay EST,RESC Paraguay CI,...,RESC Uzbekistan CI,RESC Venezuela EST,RESC Venezuela CI,RESC Vietnam EST,RESC Vietnam CI,RESC Zambia EST,RESC Zambia CI,RESC Zimbabwe EST,RESC Zimbabwe CI,Time
6,"INFLOW, CIT All, Persons, Age All",526.6,34.4,2.5,2.1,11.8,4.6,0.5,0.7,0.1,...,0.2,0.3,0.2,0.3,2.2,2,2.4,4.6,0.2,2016
7,"INFLOW, CIT All, Persons, AGQ 0-4",8.1,3.2,.,.,0.2,0.3,.,.,.,...,.,.,.,.,.,.,.,.,.,2016
8,"INFLOW, CIT All, Persons, AGQ 5-9",10.2,7,.,.,0.1,0.1,.,.,.,...,.,.,.,.,.,.,.,.,.,2016
9,"INFLOW, CIT All, Persons, AGQ 10-14",7.9,3.9,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,2016
10,"INFLOW, CIT All, Persons, AGQ 15-19",55.5,10,1.1,1.4,2.3,2.1,.,.,.,...,.,.,0.1,0.3,.,.,.,.,0.2,2016
11,"INFLOW, CIT All, Persons, AGQ 20-24",144.6,20,1,1.5,2.7,2.4,.,.,.,...,.,.,0.1,0.1,0.7,1,2.3,4.6,.,2016
12,"INFLOW, CIT All, Persons, AGQ 25-29",117.3,14.6,0.2,0.4,4,2.7,.,.,0.1,...,.,.,.,.,1.1,1.6,.,.,.,2016
13,"INFLOW, CIT All, Persons, AGQ 30-34",72.2,13.1,.,.,1.3,1.4,.,.,.,...,0.2,0.3,.,.,0.4,0.8,.,.,.,2016
14,"INFLOW, CIT All, Persons, AGQ 35-39",42.8,9,.,.,0.5,0.9,0.5,0.7,.,...,.,.,.,.,.,.,0.1,0.1,.,2016
15,"INFLOW, CIT All, Persons, AGQ 40-44",26.7,7.3,0.1,0.1,0.6,0.6,.,.,.,...,.,.,.,.,.,.,.,.,.,2016


In [15]:

"""
Flatten Everything
"""
notObsColumns = ["RESC All EST","Time"]
obsColumns = [x for x in sourceDf.columns.values if x not in notObsColumns]


flatFrames = []
for i in range(0, len(obsColumns)-1, 2):
    
    newDf = pd.DataFrame()
    
    newDf["V4_2"] = sourceDf[obsColumns[i]]
    newDf["Data_Marking"] = "" # for later
    newDf["CI"] = sourceDf[obsColumns[i+1]]
    
    newDf["Time_codelist"] = "Year"
    newDf["Time"] = sourceDf["Time"]
    
    newDf["Geography"] = "United Kingdom"
    newDf["Geography_codelist"] = "K02000001"
    
    newDf["rowVars"] = oldDf["RESC All EST"]
    newDf["colVars"] = obsColumns[i]
    
    flatFrames.append(newDf)
    
flattenedDf = pd.concat(flatFrames)

flattenedDf[-5:]


,V4_2,Data_Marking,CI,Time_codelist,Time,Geography,Geography_codelist,rowVars,colVars
1441,.,,.,Year,2016,United Kingdom,K02000001,"BALANCE, CIT Not British or British Overseas, ...",RESC Zambia CI
1442,.,,.,Year,2016,United Kingdom,K02000001,"BALANCE, CIT Not British or British Overseas, ...",RESC Zambia CI
1443,.,,.,Year,2016,United Kingdom,K02000001,"BALANCE, CIT Not British or British Overseas, ...",RESC Zambia CI
1444,.,,.,Year,2016,United Kingdom,K02000001,"BALANCE, CIT Not British or British Overseas, ...",RESC Zambia CI
1445,.,,.,Year,2016,United Kingdom,K02000001,"BALANCE, CIT Not British or British Overseas, ...",RESC Zambia CI


In [16]:

"""
Splitting out the row and col vars
"""

def codeListify(cell):
    
    cell = cell.replace(" ", "-")
    cell = cell.lower()
    return cell

    
# We have 4 comma seperated dimension items in rowVars
# Make some columns to put them in
flattenedDf["Flow"] = flattenedDf["rowVars"].astype(str).map(lambda x: x.split(",")[0].strip().title())
flattenedDf["Flow_codelist"] = flattenedDf["Flow"].apply(codeListify)

flattenedDf["Citizenship Group"] = flattenedDf["rowVars"].astype(str).map(lambda x: x.split(",")[1][4:].strip())
flattenedDf["Citizenship Group_codelist"] = flattenedDf["Citizenship Group"].apply(codeListify)

flattenedDf["Sex"] = flattenedDf["rowVars"].astype(str).map(lambda x: x.split(",")[2].strip())
flattenedDf["Sex_codelist"] = flattenedDf["Sex"].apply(codeListify)

# Temp Column
flattenedDf["AGEGROUP"] = flattenedDf["rowVars"].astype(str).map(lambda x: x.split(",")[3].strip()[:4])

flattenedDf["Age"] = flattenedDf["rowVars"].astype(str).map(lambda x: x.split(",")[3].strip()[3:].strip())
flattenedDf["Age_codelist"] = flattenedDf["Age"].apply(codeListify)

"""
Do the same for col vars
"""

flattenedDf["Country"] = flattenedDf["colVars"].astype(str).map(lambda x: x.split(" ")[1].strip().title())
flattenedDf["Country_codelist"] = flattenedDf["Country"].apply(codeListify)
 
# delete
for col in ["rowVars","colVars"]:
    flattenedDf = flattenedDf.drop(col, axis=1)

    
flattenedDf[-5:]

,V4_2,Data_Marking,CI,Time_codelist,Time,Geography,Geography_codelist,Flow,Flow_codelist,Citizenship Group,Citizenship Group_codelist,Sex,Sex_codelist,AGEGROUP,Age,Age_codelist,Country,Country_codelist
1441,.,,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,15-24,15-24,Zambia,zambia
1442,.,,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,25-44,25-44,Zambia,zambia
1443,.,,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,45-59,45-59,Zambia,zambia
1444,.,,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,60-64,60-64,Zambia,zambia
1445,.,,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,65 plus,65-plus,Zambia,zambia


In [17]:
"""
Move over data markings
"""

flattenedDf["Data_Marking"][flattenedDf["V4_2"] == "."] = '.'
flattenedDf["V4_2"][flattenedDf["V4_2"] == "."] = ''

flattenedDf[-5:]      

,V4_2,Data_Marking,CI,Time_codelist,Time,Geography,Geography_codelist,Flow,Flow_codelist,Citizenship Group,Citizenship Group_codelist,Sex,Sex_codelist,AGEGROUP,Age,Age_codelist,Country,Country_codelist
1441,,.,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,15-24,15-24,Zambia,zambia
1442,,.,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,25-44,25-44,Zambia,zambia
1443,,.,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,45-59,45-59,Zambia,zambia
1444,,.,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,60-64,60-64,Zambia,zambia
1445,,.,.,Year,2016,United Kingdom,K02000001,Balance,balance,Not British or British Overseas,not-british-or-british-overseas,Female,female,AG2,65 plus,65-plus,Zambia,zambia


In [18]:

"""
Reorder
"""

correctOrder = ['V4_2',
                'Data_Marking',
                'CI',
                'Time_codelist', 
                'Time', 'Geography', 
                'Geography_codelist',
                'Flow_codelist', 
                'Flow',
                'Citizenship Group_codelist',
                'Citizenship Group',
                'Sex_codelist',
                'Sex',
                'Age_codelist', 
                'Age',
                'Country_codelist',
                'Country', 
                'AGEGROUP']

orderedDataFrame = pd.DataFrame()

for col in correctOrder:
    orderedDataFrame[col] = flattenedDf[col]
    
orderedDataFrame[-5:]

,V4_2,Data_Marking,CI,Time_codelist,Time,Geography,Geography_codelist,Flow_codelist,Flow,Citizenship Group_codelist,Citizenship Group,Sex_codelist,Sex,Age_codelist,Age,Country_codelist,Country,AGEGROUP
1441,,.,.,Year,2016,United Kingdom,K02000001,balance,Balance,not-british-or-british-overseas,Not British or British Overseas,female,Female,15-24,15-24,zambia,Zambia,AG2
1442,,.,.,Year,2016,United Kingdom,K02000001,balance,Balance,not-british-or-british-overseas,Not British or British Overseas,female,Female,25-44,25-44,zambia,Zambia,AG2
1443,,.,.,Year,2016,United Kingdom,K02000001,balance,Balance,not-british-or-british-overseas,Not British or British Overseas,female,Female,45-59,45-59,zambia,Zambia,AG2
1444,,.,.,Year,2016,United Kingdom,K02000001,balance,Balance,not-british-or-british-overseas,Not British or British Overseas,female,Female,60-64,60-64,zambia,Zambia,AG2
1445,,.,.,Year,2016,United Kingdom,K02000001,balance,Balance,not-british-or-british-overseas,Not British or British Overseas,female,Female,65-plus,65 plus,zambia,Zambia,AG2


In [19]:
orderedDataFrame["AGEGROUP"].unique()

array(['Age ', 'AGQ ', 'AG1 ', 'AG2 '], dtype=object)

In [20]:

# Splitting up
# have to use != as all three outputs want the "Age" AgeGroup (the "all" dimension for age)

# AGQ
frame = pd.DataFrame()
frame = orderedDataFrame[orderedDataFrame["AGEGROUP"] != "AG2 "]
frame = frame[frame["AGEGROUP"] != "AG1 "]
frame = frame.drop("AGEGROUP", axis=1)
frame.to_csv("AGQ_citizenshipgroupbysexbyagebycountryoflastornextresidence_" + year + ".csv", index=False)

# AG1
frame = pd.DataFrame()
frame = orderedDataFrame[orderedDataFrame["AGEGROUP"] != "AGQ "]
frame = frame[frame["AGEGROUP"] != "AG1 "]
frame = frame.drop("AGEGROUP", axis=1)
frame.to_csv("AG2_citizenshipgroupbysexbyagebycountryoflastornextresidence_" + year + ".csv", index=False)

# AG2
frame = pd.DataFrame()
frame = orderedDataFrame[orderedDataFrame["AGEGROUP"] != "AG2 "]
frame = frame[frame["AGEGROUP"] != "AGQ "]
frame = frame.drop("AGEGROUP", axis=1)
frame.to_csv("AG1_citizenshipgroupbysexbyagebycountryoflastornextresidence_" + year + ".csv", index=False)
